In [1]:
!gcloud auth login
!mkdir /content/images/
!gsutil -mq cp gs://lily-li-dataset/satire/**.jsonl /content/
!gsutil -mq cp -r gs://lily-li-dataset/satire/defaults/images/ /content/images/

In [2]:
!pip install jsonlines tokenizers transformers

     |████████████████████████████████| 3.0MB 6.1MB/s 
     |████████████████████████████████| 778kB 56.5MB/s 
     |████████████████████████████████| 1.1MB 57.2MB/s 
     |████████████████████████████████| 890kB 57.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3e7cea2e8aa97f0be77c774a9c96609fc55770c8c292b735e44962b75bb50ba4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.8.1 which is incompatible.


In [3]:
import numpy as np
import jsonlines
from PIL import Image
import tensorflow as tf

with jsonlines.open('/content/train.jsonl', 'r') as data:
    hl_train = []
    img_train = []
    y_train = []
    for d in data:
        try:
            filepath = "/content/images/images/" + str(d["id"]) + ".jpg"  
            img = Image.open(filepath).resize((128,128), Image.BICUBIC).convert('RGB')
            img_train.append(np.asarray(img) / 255.)
            hl_train.append(d["text"])
            y_train.append(d["label"])
        except:
            print("Image at filepath {0} does not exist".format(filepath))

with jsonlines.open('/content/test.jsonl', 'r') as data:
    hl_test = []
    img_test= []
    y_test = []
    for d in data:
        try:
            filepath = "/content/images/images/" + str(d["id"]) + ".jpg"  
            img = Image.open(filepath).resize((128,128), Image.BICUBIC).convert('RGB')
            img_test.append(np.asarray(img) / 255.)
            hl_test.append(d["text"])
            y_test.append(d["label"])
        except:
            print("Image at filepath {0} does not exist".format(filepath))

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [4]:
from transformers import AutoTokenizer, TFBertModel 
max_len = 128
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_dict = tokenizer(list(hl_train), padding="max_length", truncation=True, max_length=max_len, return_tensors="tf")
train_input_ids, train_segment_ids, train_attention_masks = (train_dict["input_ids"],
                                                             train_dict["token_type_ids"],
                                                             train_dict["attention_mask"])
test_dict = tokenizer(list(hl_test), padding="max_length", truncation=True, max_length=max_len, return_tensors="tf")
test_input_ids, test_segment_ids, test_attention_masks = (test_dict["input_ids"],
                                                             test_dict["token_type_ids"],
                                                             test_dict["attention_mask"])

In [5]:
def create_model():
    encoder = TFBertModel.from_pretrained("bert-base-uncased")  

    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
    segment_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
    input_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)

    bert_output = encoder(input_ids, token_type_ids=segment_ids, attention_mask=input_mask)[0]
    text_flattened = tf.keras.layers.Flatten()(bert_output)
    text_output = tf.keras.layers.Dense(1, activation='sigmoid')(text_flattened)
    ####################################
    ResNet = tf.keras.applications.ResNet101(
            include_top=False,
            weights="imagenet",
            input_shape=(128,128,3),
            )

    input_img = tf.keras.layers.Input(shape=(128,128,3))
    resnet_output = ResNet(input_img)
    img_flattened = tf.keras.layers.Flatten()(resnet_output)
    img_output = tf.keras.layers.Dense(1, activation='sigmoid')(img_flattened)
    #########################################
    output = tf.keras.layers.Average()([text_output, img_output])

    model = tf.keras.Model(
        inputs=[input_ids, segment_ids, input_mask, input_img],
        outputs=output,
    )
    return model

In [6]:
img_train = np.stack(img_train, axis=0)
img_test = np.stack(img_test, axis=0)

img_train = tf.convert_to_tensor(img_train)
img_test = tf.convert_to_tensor(img_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [7]:
model = create_model()
model.summary()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 128, 128, 3) 0                                            
_______________________________________________________________________________________

In [8]:
optimizer = tf.keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.binary_crossentropy,
              metrics=["accuracy", "AUC", "Precision", "Recall"])

In [9]:
model.fit(x=[train_input_ids, train_segment_ids, train_attention_masks, img_train],
           y=y_train, batch_size=16, epochs=5, validation_steps=2000//32,
           steps_per_epoch=8000//32,
           validation_data=([test_input_ids, test_segment_ids, test_attention_masks, img_test], y_test))

Epoch 1/5
250/250 [==============================] - 92s 366ms/step - loss: 0.4544 - accuracy: 0.7972 - auc: 0.8644 - precision: 0.7684 - recall: 0.7044 - val_loss: 0.5543 - val_accuracy: 0.5917 - val_auc: 0.9511 - val_precision: 1.0000 - val_recall: 0.0049
Epoch 2/5
250/250 [==============================] - 88s 354ms/step - loss: 0.3428 - accuracy: 0.8830 - auc: 0.9353 - precision: 0.8739 - recall: 0.8283 - val_loss: 0.4671 - val_accuracy: 0.7823 - val_auc: 0.9376 - val_precision: 0.9614 - val_recall: 0.4889
Epoch 3/5
250/250 [==============================] - 88s 354ms/step - loss: 0.1369 - accuracy: 0.9715 - auc: 0.9949 - precision: 0.9697 - recall: 0.9588 - val_loss: 0.3565 - val_accuracy: 0.9173 - val_auc: 0.9333 - val_precision: 0.9683 - val_recall: 0.8256
Epoch 4/5
250/250 [==============================] - 89s 354ms/step - loss: 0.1357 - accuracy: 0.9688 - auc: 0.9939 - precision: 0.9671 - recall: 0.9544 - val_loss: 0.2897 - val_accuracy: 0.9304 - val_auc: 0.9618 - val_precisi